In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import torchvision
from torchsummary import summary
import math
import numpy as np
import random

In [0]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None

    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)

    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()


    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        activation1 = out
        out = self.block2(out)
        activation2 = out
        out = self.block3(out)
        activation3 = out
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out), activation1, activation2, activation3

In [3]:
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
    ])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=False)

Files already downloaded and verified


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### WRN-16-1

In [9]:
accuracies = []

for seed in ([1, 2, 3]):

  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  
  wrn_16_1 = WideResNet(16, 10)
  wrn_16_1_load = torch.load("/content/gdrive/My Drive/CIFAR10/WRN-16-1/last.pth.tar")
  wrn_16_1.load_state_dict(wrn_16_1_load['state_dict'])
  device = torch.device('cuda:0')
  wrn_16_1 = wrn_16_1.to(device)
  wrn_16_1.eval()

  with torch.no_grad():

      correct = 0
      total = 0

      for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          outputs = wrn_16_1(images)
          _, predicted = torch.max(outputs[0].data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item() 

      epoch_accuracy = round(correct / total *100, 4)
      accuracies.append(epoch_accuracy)
      
print(f'Mean accuracy is {np.mean(accuracies)} with standard deviation equal to {np.std(accuracies)}')

Mean accuracy is 91.08999999999999 with standard deviation equal to 1.4210854715202004e-14


In [12]:
accuracies = []

for seed in ([1, 2, 3]):
  
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  
  wrn_16_2 = WideResNet(16, 10, widen_factor=2)
  wrn_16_2_load = torch.load("/content/gdrive/My Drive/CIFAR10/WRN-16-2/last.pth.tar")
  wrn_16_2.load_state_dict(wrn_16_2_load['state_dict'])
  device = torch.device('cuda:0')
  wrn_16_2 = wrn_16_2.to(device)
  wrn_16_2.eval()

  with torch.no_grad():

      correct = 0
      total = 0

      for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          outputs = wrn_16_2(images)
          _, predicted = torch.max(outputs[0].data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item() 

      epoch_accuracy = round(correct / total *100, 4)
      accuracies.append(epoch_accuracy)
      
print(f'Mean accuracy is {np.mean(accuracies)} with standard deviation equal to {np.std(accuracies)}')

Mean accuracy is 94.08999999999999 with standard deviation equal to 1.4210854715202004e-14


In [15]:
accuracies = []

for seed in ([1, 2, 3]):
  
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  
  wrn_40_1 = WideResNet(40, 10)
  wrn_40_1_load = torch.load("/content/gdrive/My Drive/CIFAR10/WRN-40-1/last.pth.tar")
  wrn_40_1.load_state_dict(wrn_40_1_load['state_dict'])
  device = torch.device('cuda:0')
  wrn_40_1 = wrn_40_1.to(device)
  wrn_40_1.eval()

  with torch.no_grad():

      correct = 0
      total = 0

      for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          outputs = wrn_40_1(images)
          _, predicted = torch.max(outputs[0].data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item() 

      epoch_accuracy = round(correct / total *100, 4)
      accuracies.append(epoch_accuracy)
      
print(f'Mean accuracy is {np.mean(accuracies)} with standard deviation equal to {np.std(accuracies)}')

Mean accuracy is 93.13 with standard deviation equal to 0.0


In [16]:
accuracies = []

for seed in ([1, 2, 3]):
  
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  
  wrn_40_2 = WideResNet(40, 10, widen_factor=2)
  wrn_40_2_load = torch.load("/content/gdrive/My Drive/CIFAR10/WRN-40-2/last.pth.tar")
  wrn_40_2.load_state_dict(wrn_40_2_load['state_dict'])
  device = torch.device('cuda:0')
  wrn_40_2 = wrn_40_2.to(device)
  wrn_40_2.eval()

  with torch.no_grad():

      correct = 0
      total = 0

      for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          outputs = wrn_40_2(images)
          _, predicted = torch.max(outputs[0].data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item() 

      epoch_accuracy = round(correct / total *100, 4)
      accuracies.append(epoch_accuracy)
      
print(f'Mean accuracy is {np.mean(accuracies)} with standard deviation equal to {np.std(accuracies)}')

Mean accuracy is 94.71999999999998 with standard deviation equal to 1.4210854715202004e-14
